In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd
from collections import OrderedDict
from shapely.geometry import Point, LineString

import scalebar

import nept
import scalebar

from loading_data import get_data
from utils_maze import get_xyedges, find_zones, speed_threshold
from analyze_decode import point_in_zones

from utils_fields import categorize_fields
from analyze_tuning_curves import get_odd_firing_idx
from plot_sequence_raster import plot_sequence

In [ ]:
output_filepath = 'E:/code/emi_shortcut/plots/intermediate'
pickle_filepath = 'E:/code/emi_shortcut/cache/pickled'

# output_filepath = 'C:/Users/Emily/code/emi_shortcut/plots/intermediate'
# pickle_filepath = 'C:/Users/Emily/code/emi_shortcut/cache/pickled'

In [ ]:
import info.r066d3 as info

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)

zones = find_zones(info, remove_feeder=True, expand_by=8)

experiment_time = 'phase1'
filename = '_decode-' + experiment_time + '.pkl'
decode_filename = info.session_id + filename
pickled_decoded = os.path.join(pickle_filepath, decode_filename)

with open(pickled_decoded, 'rb') as fileobj:
    decode = pickle.load(fileobj)

In [ ]:
zones = find_zones(info, remove_feeder=True, expand_by=6)

novel_idx = []
shortcut_idx = []
u_idx = []
pedestal_idx = []
other_idx = []

for i, (x, y) in enumerate(zip(position.x, position.y)):
    point = Point([x, y])
    if zones['novel'].contains(point):
        novel_idx.append(i)
        continue
    elif zones['shortcut'].contains(point):
        shortcut_idx.append(i)
        continue
    elif zones['u'].contains(point):
        u_idx.append(i)
        continue
    elif zones['pedestal'].contains(point):
        pedestal_idx.append(i)
        continue
    else:
        other_idx.append(i)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
ax.plot(position.x, position.y, '.', color='k', ms=1)
ax.plot(position.x[u_idx], position.y[u_idx], '.', color='#0072b2', ms=2)
ax.plot(position.x[shortcut_idx], position.y[shortcut_idx], '.', color='#009e73', ms=2)
ax.plot(position.x[novel_idx], position.y[novel_idx], '.', color='#d55e00', ms=2)

ax.set_yticklabels([])
ax.set_xticklabels([])
ax.axis('off')

# savepath = os.path.join(output_filepath, 'recording_trajectories.png')
# plt.savefig(savepath, bbox_inches='tight', transparent=True)

plt.show()

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(decode['decoded'].x, decode['decoded'].y, bins=xx.shape)

cmap = plt.cm.get_cmap('bone_r')
pp = plt.pcolormesh(yy, xx, histogram, vmin=0., vmax=50., cmap=cmap)
plt.colorbar(pp)
plt.axis('off')
# plt.show()

savepath = os.path.join(output_filepath, 'r063d6_decoded.png')
plt.savefig(savepath, bbox_inches='tight', transparent=True)

In [ ]:
for idx in range(10):
    start = nept.find_nearest_idx(decode['decoded'].time, decode['epochs'].starts[idx])
    stop = nept.find_nearest_idx(decode['decoded'].time, decode['epochs'].stops[idx])

    xx, yy = np.meshgrid(xedges, yedges)
    histogram, xs, ys = np.histogram2d(decode['decoded'].x[start:stop], decode['decoded'].y[start:stop], bins=xx.shape)

    cmap = plt.cm.get_cmap('bone_r', 25)
    pp = plt.pcolormesh(yy, xx, histogram, cmap=cmap)
    plt.colorbar(pp)
    # plt.axis('off')
    plt.show()

In [ ]:
neurons_filename = info.session_id + '_neurons.pkl'
pickled_neurons = os.path.join(pickle_filepath, neurons_filename)
with open(pickled_neurons, 'rb') as fileobj:
    neurons = pickle.load(fileobj)

In [ ]:
multiple_tuning_curves = np.zeros(neurons.tuning_shape)
multiple_tuning_curves += neurons.tuning_curves[28]
multiple_tuning_curves += neurons.tuning_curves[11]
multiple_tuning_curves += neurons.tuning_curves[10]
multiple_tuning_curves += neurons.tuning_curves[4]

plt.figure(figsize=(6, 5))
pp = plt.pcolormesh(xx, yy, multiple_tuning_curves, cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.tight_layout()
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-28.png'))
# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-28-11.png'))
# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-28-11-10.png'))
# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-28-11-10-4.png'))
# plt.close()

In [ ]:
all_tuning_curves = np.zeros(neurons.tuning_shape)
for tc in neurons.tuning_curves:
    all_tuning_curves += tc

plt.figure(figsize=(6, 5))
pp = plt.pcolormesh(xx, yy, all_tuning_curves, vmin=100., vmax=400., cmap='bone_r')
# plt.colorbar(pp)
plt.axis('off')
plt.show()

# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-all.png'))
# plt.close()

In [ ]:
experiment_time = 'phase3'
shuffle_id = False
window_size=0.025
window_advance=0.025
speed_limit=0.4
min_swr=3
sequence_speed=5
sequence_len=3
min_epochs=3

In [ ]:
track_times = ['phase1', 'phase2', 'phase3', 'tracks']
pedestal_times = ['pauseA', 'pauseB', 'prerecord', 'postrecord']

events, position, all_spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)

exp_start = info.task_times[experiment_time].start
exp_stop = info.task_times[experiment_time].stop

sliced_spikes = neurons.time_slice(exp_start, exp_stop)
sliced_spikes = sliced_spikes.spikes

if experiment_time in track_times:
    run_position = run_position[speed_threshold(position, speed_limit=speed_limit)]
else:
    run_position = position

exp_position = run_position.time_slice(exp_start, exp_stop)

if shuffle_id:
    tuning_curves = np.random.permutation(neurons.tuning_curves)
else:
    tuning_curves = neurons.tuning_curves

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(exp_position.x, exp_position.y, bins=xx.shape)

cmap = plt.cm.get_cmap('bone_r')
pp = plt.pcolormesh(yy, xx, histogram, cmap=cmap)
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
if experiment_time in track_times:
    epochs_interest = nept.Epoch(np.hstack([exp_start, exp_stop]))
elif experiment_time in pedestal_times:
    sliced_lfp = lfp.time_slice(exp_start, exp_stop)

    z_thresh = 3.0
    power_thresh = 5.0
    merge_thresh = 0.02
    min_length = 0.05
    swrs = nept.detect_swr_hilbert(sliced_lfp, fs=info.fs, thresh=(140.0, 250.0), z_thresh=z_thresh,
                                   power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)

    min_involved = 4
    multi_swr = nept.find_multi_in_epochs(sliced_spikes, swrs, min_involved=min_involved)

    if multi_swr.n_epochs < min_swr:
        epochs_interest = nept.Epoch(np.array([[], []]))
    else:
        epochs_interest = multi_swr

    print('sharp-wave ripples, total:', swrs.n_epochs)
    print('sharp-wave ripples, min', min_involved, 'neurons :', multi_swr.n_epochs)
    print('sharp-wave ripples, used :', epochs_interest.n_epochs)
    print('sharp-wave ripples, mean durations: ', np.mean(epochs_interest.durations))

else:
    raise ValueError("unrecognized experimental phase. Must be in ['prerecord', 'phase1', 'pauseA', 'phase2', "
                     "'pauseB', phase3', 'postrecord'].")

time_edges = nept.get_edges(exp_position.time, window_advance, lastbin=True)
counts = nept.bin_spikes(sliced_spikes, exp_position.time, dt=window_advance, window=window_size,
                         gaussian_std=None, normalized=False)

tc_shape = tuning_curves.shape
decoding_tc = tuning_curves.reshape(tc_shape[0], tc_shape[1] * tc_shape[2])

min_neurons = 2
min_spikes = 1
likelihood = nept.bayesian_prob(counts, decoding_tc, window_size, min_neurons=min_neurons, min_spikes=min_spikes)

In [ ]:
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = nept.cartesian(xcenters, ycenters)

decoded = nept.decode_location(likelihood, xy_centers, time_edges)
nan_idx = np.logical_and(np.isnan(decoded.x), np.isnan(decoded.y))
decoded = decoded[~nan_idx]

sequences = nept.remove_teleports(decoded, speed_thresh=sequence_speed, min_length=sequence_len)
decoded_epochs = epochs_interest.intersect(sequences)
decoded_epochs = decoded_epochs.expand(0.05)

if decoded_epochs.n_epochs < min_epochs:
    decoded = nept.Position(np.array([[], []]), np.array([]))
else:
    decoded = decoded[decoded_epochs]

actual_x = np.interp(decoded.time, exp_position.time, exp_position.x)
actual_y = np.interp(decoded.time, exp_position.time, exp_position.y)
actual_position = nept.Position(np.hstack((actual_x[..., np.newaxis],
                                           actual_y[..., np.newaxis])), decoded.time)
errors = actual_position.distance(decoded)

In [ ]:
histogram, xs, ys = np.histogram2d(actual_position.x, actual_position.y, bins=xx.shape)

pp = plt.pcolormesh(yy, xx, histogram, vmin=0., vmax=350., cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
zones = find_zones(info, remove_feeder=True, expand_by=8)
decoded_zones = point_in_zones(decoded, zones)

keys = ['u', 'shortcut', 'novel']

errors = dict()
actual_pos = dict()
if experiment_time in ['phase1', 'phase2', 'phase3', 'tracks']:
    for trajectory in keys:
        actual_x = np.interp(decoded_zones[trajectory].time, exp_position.time, exp_position.x)
        actual_y = np.interp(decoded_zones[trajectory].time, exp_position.time, exp_position.y)
        actual_pos[trajectory] = nept.Position(np.hstack((actual_x[..., np.newaxis],
                                                               actual_y[..., np.newaxis])),
                                                    decoded_zones[trajectory].time)
        if actual_pos[trajectory].n_samples > 0:
            errors[trajectory] = actual_pos[trajectory].distance(decoded_zones[trajectory])
else:
    for trajectory in decoded_zones:
        errors[trajectory] = []
        actual_pos[trajectory] = []

In [ ]:
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = nept.cartesian(xcenters, ycenters)

field_thresh = 1.0
fields_tunings = categorize_fields(neurons.tuning_curves, zones, xedges, yedges, 
                                   field_thresh=field_thresh)

u_line = LineString(info.u_trajectory)

u_dist = []
for neuron in fields_tunings['u']:
    yy = ycenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[0][0]]
    xx = xcenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[1][0]]

    pt = Point(xx, yy)
    if zones['u'].contains(pt):
        u_dist.append((u_line.project(pt), neuron))
        
ordered_dist_u = sorted(u_dist, key=lambda x:x[0])
sort_idx = []
for neuron in ordered_dist_u:
    sort_idx.append(neuron[1])
    
sort_spikes = []
sort_tuning_curves = []
for neuron in sort_idx:
    sort_tuning_curves.append(fields_tunings['u'][neuron])
    sort_spikes.append(neurons.spikes[neuron])
    
odd_firing_idx = get_odd_firing_idx(sort_tuning_curves, max_mean_firing=2)

ordered_spikes = []
ordered_fields =[]
for i, neuron in enumerate(sort_spikes):
    if i not in odd_firing_idx:
        ordered_spikes.append(neuron)
        ordered_fields.append(sort_tuning_curves[i])

In [ ]:
for idx in range(10):
    plot_sequence(ordered_spikes,
                  decode['epochs'].starts[idx], 
                  decode['epochs'].stops[idx],
                  ms_fraction=300,
                  lfp=lfp,
                  position=None)

In [ ]:
errors['u'].shape

In [ ]:
error_hist_byactual = np.zeros((len(yedges), len(xedges)))
n_hist_byactual = np.ones((len(yedges), len(xedges)))
error_x = []
error_y = []

for error, x, y in zip(errors, actual_position.x, actual_position.y):
    x_idx = nept.find_nearest_idx(xedges, x)
    y_idx = nept.find_nearest_idx(yedges, y)
    error_hist_byactual[y_idx][x_idx] += error
    n_hist_byactual[y_idx][x_idx] += 1

error_byactual = error_hist_byactual / n_hist_byactual

In [ ]:
pp = plt.pcolormesh(xx, yy, error_hist_byactual, vmin=0., vmax=500., cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, n_hist_byactual, vmin=0., vmax=500., cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, error_byactual, vmin=0., vmax=50., cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

# filename = 'error_decoded.png'
# plt.savefig(os.path.join(output_filepath, filename), transparent=True)
# plt.close()

In [ ]:
error_hist_bydecode = np.zeros((len(yedges), len(xedges)))
n_hist_bydecode = np.ones((len(yedges), len(xedges)))

error_x = []
error_y = []
for error, x, y in zip(errors, decoded.x, decoded.y):
    x_idx = nept.find_nearest_idx(xedges, x)
    y_idx = nept.find_nearest_idx(yedges, y)
    error_hist_bydecode[y_idx][x_idx] += error
    n_hist_bydecode[y_idx][x_idx] += 1

error_bydecode = error_hist_bydecode / n_hist_bydecode

In [ ]:
pp = plt.pcolormesh(xx, yy, error_hist_bydecode, vmin=0., vmax=10000., cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, n_hist_bydecode, vmin=0., vmax=500., cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, error_bydecode, vmin=0., vmax=50., cmap='bone_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

# filename = 'error_decoded.png'
# plt.savefig(os.path.join(output_filepath, filename), transparent=True)
# plt.close()

In [ ]:
decoded.x

In [ ]:
fig = plt.figure(figsize=(6, 3))
# ax = plt.subplot(111)
plt.plot(decoded.time, decoded.x, '#9970ab', ms=1)
plt.plot(position.time, position.x, 'k', ms=3)
plt.xlim(25250, 25400)
# scalebar.add_scalebar(ax, matchy=False, bbox_transform=fig.transFigure,
#                       bbox_to_anchor=(0.9, 0.1), units='ms')
plt.axis('off')
plt.tight_layout()
# plt.show()
plt.savefig(os.path.join(output_filepath, info.session_id + '-decoded-x.png'), transparent=True)
plt.close()

In [ ]:
fig = plt.figure(figsize=(6, 3))
ax = plt.subplot(111)
plt.plot(decoded.time, decoded.y, '#5aae61', ms=1)
plt.plot(position.time, position.y, 'k', ms=3)
# plt.xlim(6650, 6700)
plt.xlim(25250, 25400)
scalebar.add_scalebar(ax, matchy=False, bbox_transform=fig.transFigure,
                         bbox_to_anchor=(0.9, 0.1), units='s')
plt.axis('off')
# plt.show()
plt.savefig(os.path.join(output_filepath, info.session_id + '-decoded-y.png'), transparent=True)
plt.close()

In [ ]:
os.path.join(output_filepath, info.session_id)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd
from collections import OrderedDict
from shapely.geometry import Point, LineString

import nept

from loading_data import get_data
from utils_maze import get_xyedges, find_zones, speed_threshold
# from analyze_decode import get_edges, point_in_zones

# output_filepath = 'E:/code/emi_shortcut/plots/intermediate'
# pickle_filepath = 'E:/code/emi_shortcut/cache/pickled'

from utils_fields import categorize_fields
from analyze_tuning_curves import get_odd_firing_idx
from plot_sequence_raster import plot_sequence

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)
zones = find_zones(info, remove_feeder=True, expand_by=8)

neurons_filename = info.session_id + '_neurons.pkl'
pickled_neurons = os.path.join(pickle_filepath, neurons_filename)
with open(pickled_neurons, 'rb') as fileobj:
    neurons = pickle.load(fileobj)
    
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = nept.cartesian(xcenters, ycenters)

In [ ]:
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = nept.cartesian(xcenters, ycenters)

field_thresh = 1.0
fields_tunings = categorize_fields(neurons.tuning_curves, zones, xedges, yedges, 
                                   field_thresh=field_thresh)

u_line = LineString(info.u_trajectory)

u_dist = []
for neuron in fields_tunings['u']:
    yy = ycenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[0][0]]
    xx = xcenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[1][0]]

    pt = Point(xx, yy)
    if zones['u'].contains(pt):
        u_dist.append((u_line.project(pt), neuron))
        
ordered_dist_u = sorted(u_dist, key=lambda x:x[0])
sort_idx = []
for neuron in ordered_dist_u:
    sort_idx.append(neuron[1])
    
sort_spikes = []
sort_tuning_curves = []
for neuron in sort_idx:
    sort_tuning_curves.append(fields_tunings['u'][neuron])
    sort_spikes.append(neurons.spikes[neuron])
    
odd_firing_idx = get_odd_firing_idx(sort_tuning_curves, max_mean_firing=2)

ordered_spikes = []
ordered_fields =[]
for i, neuron in enumerate(sort_spikes):
    if i not in odd_firing_idx:
        ordered_spikes.append(neuron)
        ordered_fields.append(sort_tuning_curves[i])
        
# plot_sequence(ordered_spikes,
#               decode['epochs'].starts[0], 
#               decode['epochs'].stops[0],
#               ms_fraction=10,
#               lfp=lfp,
#               position=None)

In [ ]:
plt.plot(position.x, position.y, 'k.', ms=1)
plt.show()

In [ ]:
seq = 1
times = 'swr'
filename = info.session_id + '_swr.png'
plot_sequence(ordered_spikes,
              info.sequence['u'][times].starts[seq], 
              info.sequence['u'][times].stops[seq],
              ms_fraction=250,
              lfp=lfp,
              position=None,
              savepath=None)

In [ ]:
seq = 1
times = 'run'
filename = info.session_id + '_run.png'
plot_sequence(ordered_spikes,
              info.sequence['u'][times].starts[seq], 
              info.sequence['u'][times].stops[seq],
              ms_fraction=150,
              lfp=None,
              position=position,
              savepath=None)

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)

all_tuning_curves = np.zeros(neurons.tuning_shape)
for i in range(neurons.n_neurons):
    all_tuning_curves += neurons.tuning_curves[i]

pp = plt.pcolormesh(xx, yy, all_tuning_curves, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id+'_tuning.png'))

In [ ]:
fig = plt.figure(figsize=(4, 3))
ax = plt.subplot(111)
start = info.sequence['u']['swr'].starts[1]
stop = info.sequence['u']['swr'].stops[1]
start_idx = nept.find_nearest_idx(lfp.time, start)
stop_idx = nept.find_nearest_idx(lfp.time, stop)
plt.plot(lfp.time[start_idx:stop_idx], lfp.data[start_idx:stop_idx], 'k', lw=0.8)
plt.axis('off')
scalebar.add_scalebar(ax, matchy=False, bbox_transform=fig.transFigure, bbox_to_anchor=(0.9, 0.2), units='ms')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-swr1.png'), transparent=True)
# plt.close()

In [ ]:
fig = plt.figure(figsize=(4, 3))
ax = plt.subplot(111)
start = info.sequence['u']['swr'].starts[0]
stop = info.sequence['u']['swr'].stops[0]
start_idx = nept.find_nearest_idx(lfp.time, start)
stop_idx = nept.find_nearest_idx(lfp.time, stop)
plt.plot(lfp.time[start_idx:stop_idx], lfp.data[start_idx:stop_idx], 'k', lw=0.8)
plt.axis('off')
scalebar.add_scalebar(ax, matchy=False, bbox_transform=fig.transFigure, bbox_to_anchor=(0.9, 0.1), units='ms')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-swr2.png'), transparent=True)
# plt.close()

In [ ]:
# experiment_time = 'phase2'
exp_start = info.sequence['u']['run'].starts[1]
exp_stop = info.sequence['u']['run'].stops[1]

sliced_spikes = neurons.time_slice(exp_start, exp_stop)
sliced_spikes = sliced_spikes.spikes

speed_limit = 0.4
run_position = position[speed_threshold(position, speed_limit=speed_limit)]
exp_position = run_position.time_slice(exp_start, exp_stop)

In [ ]:
field_thresh = 1.0
fields_tunings = categorize_fields(neurons.tuning_curves, zones, xedges, yedges, 
                                   field_thresh=field_thresh)

u_line = LineString(info.u_trajectory)

u_dist = []
for neuron in fields_tunings['u']:
    yy = ycenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[0][0]]
    xx = xcenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[1][0]]

    pt = Point(xx, yy)
    if zones['u'].contains(pt):
        u_dist.append((u_line.project(pt), neuron))
        
ordered_dist_u = sorted(u_dist, key=lambda x:x[0])
sort_idx = []
for neuron in ordered_dist_u:
    sort_idx.append(neuron[1])
    
sort_spikes = []
sort_tuning_curves = []
for neuron in sort_idx:
    sort_tuning_curves.append(fields_tunings['u'][neuron])
    sort_spikes.append(sliced_spikes[neuron])
    
odd_firing_idx = get_odd_firing_idx(sort_tuning_curves, max_mean_firing=2)

ordered_spikes = []
ordered_fields =[]
for i, neuron in enumerate(sort_spikes):
    if i not in odd_firing_idx:
        ordered_spikes.append(neuron)
        ordered_fields.append(sort_tuning_curves[i])

In [ ]:
window_size = 0.025
window_advance = 0.025
time_edges = nept.get_edges(exp_position.time, window_advance, lastbin=True)
counts = nept.bin_spikes(ordered_spikes, exp_position.time, dt=window_advance, window=window_size,
                             gaussian_std=None, normalized=True)
# counts = nept.get_counts(ordered_spikes, time_edges, gaussian_std=0.005)

In [ ]:
start = info.sequence['u']['run'].starts[1] 
stop = info.sequence['u']['run'].stops[1] 

plt.rc('xtick', labelsize=16) 
plt.rc('ytick', labelsize=16) 

fig = plt.figure(figsize=(5, 7))
ax = plt.subplot(111)
pp = plt.pcolormesh(counts.data.T, cmap='bone_r')
# pp = plt.pcolormesh(counts.time_slice(start, stop).data.T, cmap='bone_r')
plt.colorbar(pp)
ax.set_xticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.get_yaxis().tick_left()
# plt.show()
plt.savefig(os.path.join(output_filepath, info.session_id + '-binned-spikes.png'), transparent=True)
plt.close()

In [ ]:
spike_loc = 1
rows = len(ordered_spikes)
ms_fraction = 150
ms=20
mew = 0.5

fig = plt.figure(figsize=(4.5, 6))
ax = plt.subplot(111)
for idx, neuron_spikes in enumerate(ordered_spikes):
    ax.plot(neuron_spikes.time, np.ones(len(neuron_spikes.time))+(idx*spike_loc), '|',
             color='k', ms=ms, mew=mew)
ax.set_xticks([])
ax.set_xlim([start, stop])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()
plt.tight_layout()
# plt.show()
plt.savefig(os.path.join(output_filepath, info.session_id + '-raw-spikes.png'), transparent=True)
plt.close()

In [ ]:
print(info.sequence['u']['run'].starts[1], info.sequence['u']['run'].stops[1])

seq = 1
times = 'run'
filename = info.session_id + '_run.png'
plot_sequence(ordered_spikes,
              info.sequence['u'][times].starts[seq], 
              info.sequence['u'][times].stops[seq],
              ms_fraction=150,
              lfp=None,
              position=position)

In [ ]:
start_idx, stop_idx

In [ ]:
counts